### Liqudation + Inflow strategy
   * Glassnode inflow data
   * Tardis liquidation data

In [5]:
from cdcqr.data.glassnode import glassnode_data as gnd
from cdcqr.ct.utils import plot2
import pandas as pd
import cdcqr.analytics.utils as autil 
from importlib import reload
pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
import sys
sys.path.append('/core/github/cryptoderiv-quant/')
from ct.alfafactory import runsignal
import numpy as np
import os
from cdcqr.common.utils import LOCAL_DATA_DIR,load_df
%matplotlib inline
from functools import reduce

In [6]:
gn = gnd.GlassnodeData()

### download balance exchange data

In [28]:
fs =  ['balance_exchanges']
a = ['LINK','CRV','SUSHI','AAVE','FTT','MKR','CRO','MATIC']
df = gn.load_features(fs,a)

loading distribution/balance_exchanges asset=LINK, resolution=10m
loading distribution/balance_exchanges asset=CRV, resolution=10m
loading distribution/balance_exchanges asset=SUSHI, resolution=10m
loading distribution/balance_exchanges asset=AAVE, resolution=10m
loading distribution/balance_exchanges asset=FTT, resolution=10m
loading distribution/balance_exchanges asset=MKR, resolution=10m
loading distribution/balance_exchanges asset=CRO, resolution=10m
loading distribution/balance_exchanges asset=MATIC, resolution=10m


In [29]:
for col in df.columns:
    asset = col.split('-')[0]
    dfi = df[col].to_frame()
    dfi.columns=['v']
    dfi.save('glass_balexch_{}_10m_20210101_20220131'.format(asset))

saved df to C:\core\tmp\glass_balexch_LINK_10m_20210101_20220131.pickle
saved df to C:\core\tmp\glass_balexch_CRV_10m_20210101_20220131.pickle
saved df to C:\core\tmp\glass_balexch_SUSHI_10m_20210101_20220131.pickle
saved df to C:\core\tmp\glass_balexch_AAVE_10m_20210101_20220131.pickle
saved df to C:\core\tmp\glass_balexch_FTT_10m_20210101_20220131.pickle
saved df to C:\core\tmp\glass_balexch_MKR_10m_20210101_20220131.pickle
saved df to C:\core\tmp\glass_balexch_CRO_10m_20210101_20220131.pickle
saved df to C:\core\tmp\glass_balexch_MATIC_10m_20210101_20220131.pickle


In [32]:
for asset in ['LINK','CRV','SUSHI','AAVE','MKR','MATIC']:
   
    signalid = 'short-{}liq'.format(asset)

    price0 = pd.read_pickle('/core/tmp/ppt_prc_{}_1m_20210101_20220228.pkl'.format(asset))
    liq0 = pd.read_pickle('/core/tmp/ppt_liqb_{}_1m_20210101_20220228.pkl'.format(asset))

    df = pd.read_pickle('/core/tmp/glass_balexch_{}_10m_20210101_20220131.pickle'.format(asset))
    print(price0.shape, liq0.shape, df.shape)

(735651, 5) (36947, 5) (75196, 1)
(735099, 5) (22657, 5) (75196, 1)
(735557, 5) (25437, 5) (75196, 1)
(713557, 5) (17049, 5) (75196, 1)
(735377, 5) (7380, 5) (75196, 1)
(695701, 5) (39997, 5) (75196, 1)


In [22]:
dfi.columns=['v']

In [23]:
dfi

,v
t,
2020-10-03 02:20:00,1.377851e+09
2020-10-03 02:30:00,1.377851e+09
2020-10-03 02:40:00,1.377851e+09
2020-10-03 02:50:00,1.377853e+09
2020-10-03 03:00:00,1.377853e+09
2020-10-03 03:10:00,1.377841e+09
2020-10-03 03:20:00,1.377827e+09
2020-10-03 03:30:00,1.379089e+09
2020-10-03 03:40:00,1.379079e+09


### Data Overview

In [ ]:
df1 = gn.get_meta_data().query('category=="derivatives"')
liq_fs = df1[df1['f'].str.contains('liq')]['f'].values.tolist()
liq_fs

In [ ]:
gn.get_feature_category('TransfersVolumeToExchangesSum')

In [ ]:
glassmeta = gn.get_meta_data()

In [ ]:
glassmeta['category'].unique()

In [ ]:
glassmeta.query('category=="transactions"')

### BTC

In [ ]:
fs =  liq_fs+['price_usd_close']

a = ['BTC']
#i = '10m'
df = gn.load_features(fs,a)
    #display(df.shape)

In [ ]:
df.describe()

In [ ]:
df['BTC-futures_liquidated_volume_long_relative'].quantile(0.6)

In [ ]:
df['BTC-futures_liquidated_volume_long_relative'].quantile(0.4)

In [ ]:
df.iloc[:,1:2].plot()

In [ ]:
df.iloc[:,2:3].plot()

In [ ]:
df.iloc[:,3:4].plot()

In [ ]:
df1 = df[df.index>='20210101']
display(df1.head(2))
df1.plot2(cols[0],cols[1:3])

In [ ]:
reload(autil)
feature_raw = '{}_futures_funding_rate_perpetual'.format(a)
feature_normalized = feature_raw
target = '{}_price_usd_close'.format(a)
_ = autil.ts_eda(df1, feature_raw, feature_normalized, target, save_fig=True)

### Alts

In [ ]:
## load data
sym = 'AAVE'
price0 = pd.read_pickle('/core/tmp/ppt_prc_{}_1m_20210101_20220228.pkl'.format(sym))
liq0 = pd.read_pickle('/core/tmp/ppt_liqb_{}_1m_20210101_20220228.pkl'.format(sym))

In [3]:
fs = ['transfers_volume_to_exchanges_mean']

a = ['BTC']
#i = '10m'
df = gn.load_features(fs,a)
    #display(df.shape)

loading transactions/transfers_volume_to_exchanges_mean asset=BTC, resolution=10m


In [ ]:
gn.

In [5]:
df.save('transfers_volume_to_exchanges_mean')

saved df to C:\core\tmp\transfers_volume_to_exchanges_mean.pickle


In [ ]:
glass = df.copy().resample("1Min").last().fillna(0)
glass.columns = ['v']
glass

In [3]:
df = pd.read_pickle('/core/tmp/glass_balexch_BTC_10m_20210101_20220131.pkl')

In [ ]:
df

In [ ]:
def getsignal3(glass, price0, liq0, qtl=0.99, liql=20, priceq=0.0, qtl_w=0):
    glass['99q'] = glass['v'].rolling(60*24).quantile(qtl)
    glass['signal'] = (glass['v']>=glass['99q'])
    price0['glass_signal'] = glass['signal']
    price0['glass_signal'] = price0['glass_signal'].fillna(False)
    
    liq = liq0.resample("1Min").last().fillna(0)
    liq['spikes'] = liq[liq.v>liql]['v']
    liq['spikes'] = liq['spikes'].fillna(0).rolling(100).max().fillna(0)
    price0['liq_signal'] = liq['spikes'][liq['spikes']>liql]
    price0['liq_signal'] = price0['liq_signal'].fillna(False)
    
    if qtl_w == 0:
        price0['cq_signal'] = True
    else:
        price0['cq'] = pricewindow['c'].rolling(qtl_w).quantile(priceq).resample("1Min").last().ffill()
        price0['cq_signal'] = (price0['c'] >= price0['cq']).fillna(False)
#        price0['cq'] = price0['c'].rolling(60*24*qtl_w).quantile(priceq)
#        price0['cq_signal'] = (price0['c'] > price0['cq']).fillna(False)
    
    price0['s'] = price0['glass_signal'] & price0['liq_signal'] & price0['cq_signal']
    
    return price0[['c','glass_signal','liq_signal','cq_signal','s']]

In [ ]:
s = getsignal3(glass, price0, liq0)

In [ ]:
s

In [ ]:
respq=[]
price = price0
counter0 = 0
# for signalid, signal in signals:
#   signal = getsignal()['s']
for qtl in [0.99, 0.95]: #[0.995, 0.99, 0.95, 0.9]:
    for liql in [20]: #[5, 10, 20, 50]:
        for priceq in [0.0, 0.5, 0.6, 0.7, 0.8, 0.9]: #, 0.75, 0.9]:
            for qtlw in [0, 7, 30, 60, 90]:
                signal = getsignal3(qtl,liql,priceq,qtlw)['s']
            signalid = f'{qtl}-{liql}-{priceq}-{qtlw}'
            pricesignal = [[1,price.loc['2021.1.1':'2021.6.15'],signal.loc['2021.1.1':'2021.6.15']],
                           [2,price.loc['2021.6.16':'2021.11.21'],signal.loc['2021.06.16':'2021.11.21']]
                          ]
            for short in [True]:
                for j, priceseg, signalseg in pricesignal:
#                for nn1 in [1,5,10,20,50,100]:
#                    for nn2 in [101,200,300,500,1000,2000]:
                        #signal=glass.rolling(100).max()>200
                        counter0 = counter0 + 1
                        print(f'{counter0} : pq:{priceq} qtlw:{qtlw} id:{signalid} {j}')
                        ressig=runsignal(priceseg,signal=signalseg,tp=3,sl=1,ts=5,n1=None,n2=None,rsil=30,rsih=50,rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf'])
                        for k in ressig:
                            try:
                                resd={"j":j,"signalid":signalid,"qtl":qtl,"liql":liql,"priceq":priceq,"pqtlw":qtlw,"short":short,'k':k,'sr':ressig[k].sharpe_ratio(),'tr':ressig[k].total_return(),'ntrades':ressig[k].trades.count()/len(priceseg)*7*24*60}
                            except:
                                pass
                            respq.append(resd)

In [ ]:
runsignal?

In [ ]:
zip(runsignal.func_code.co_varnames, args)

In [ ]:
import inspect 
inspect.signature(runsignal).parameters.keys()

In [ ]:
[k for k in inspect.signature(runsignal).parameters.keys()]